In [22]:
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patches as mpatches
import pandas as pd
import scipy.stats as st
import numpy as np
import os
import sklearn.metrics as met
import matplotlib as mpl
import itertools as it
import math
from scipy import spatial as sp
import matplotlib.image as mpimg


mpl.use('Agg')

WHR2020 = "../WHR2020_Data.csv"
WHR2021 = "../WHR2021_Data.csv"

data20 = pd.read_csv(WHR2020)
data20 = data20.set_index('Country name')
data21 = pd.read_csv(WHR2021)
data21 = data21.set_index('Country name')

## Let's preview the data from the 2020 and 2021 reports.
### Both reports use data from the previous 3 years.

In [23]:
#2020
data20.head()

,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
Country name,,,,,,,,,,,,,,,,,,,
Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [24]:
#2021
data21.tail()

,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
Country name,,,,,,,,,,,,,,,,,,,
Lesotho,Sub-Saharan Africa,3.512,0.120,3.748,3.276,7.926,0.787,48.700,0.715,-0.131,0.915,2.43,0.451,0.731,0.007,0.405,0.103,0.015,1.800
Botswana,Sub-Saharan Africa,3.467,0.074,3.611,3.322,9.782,0.784,59.269,0.824,-0.246,0.801,2.43,1.099,0.724,0.340,0.539,0.027,0.088,0.648
Rwanda,Sub-Saharan Africa,3.415,0.068,3.548,3.282,7.676,0.552,61.400,0.897,0.061,0.167,2.43,0.364,0.202,0.407,0.627,0.227,0.493,1.095
Zimbabwe,Sub-Saharan Africa,3.145,0.058,3.259,3.030,7.943,0.750,56.201,0.677,-0.047,0.821,2.43,0.457,0.649,0.243,0.359,0.157,0.075,1.205
Afghanistan,South Asia,2.523,0.038,2.596,2.449,7.695,0.463,52.493,0.382,-0.102,0.924,2.43,0.370,0.000,0.126,0.000,0.122,0.010,1.895


## The data is almost clean.
### We're going to drop any countries that do not appear in both reports.

In [25]:
# Begin by initializing two empty lists for countries that are not in both reports
    # Either a country was dropped from the report in 2021 or a country was added to the report in 2021
lost_country = []
gain_country = []

# Find countries that were in the 2020 report and dropped in 2021
for c in data20.index:
    if c not in data21.index:
        lost_country.append(c)

# Drop these countries from the whole dataset
data20 = data20.drop(lost_country)

# Find countries that were not in the 2020 report and were added to the 2021 report 
for c in data21.index:
    if c not in data20.index:
        gain_country.append(c)



# Again, redefine the data without that country
data21 = data21.drop(gain_country)

## We are going to group our data by regions.

In [26]:
by_region20 = data20.groupby(by = 'Regional indicator')
by_region21 = data21.groupby(by = 'Regional indicator')

In [27]:
region_list20 = data20['Regional indicator'].drop_duplicates().to_list()
region_list21 = data21['Regional indicator'].drop_duplicates().to_list()

# Make sure both reports consider the same regions
region_list20 == region_list21

True

In [28]:
# Regions across the two reports are the same, so we only need one list
region_list = region_list20

## The report is concerned mainly with the average ladder score for each country
### We're going to gather some summary statistics of the ladder score by region

In [29]:
#We're going to look at most of the standard sample statistics
    # We're going to include the standard mean error and the skewness
    
ladder_stats20 = by_region20['Ladder score'].agg(['count','mean', 'sem', 'min', 'max', 'median', 'var', 'std', 'skew'])
ladder_stats20.sort_values('mean',ascending=False)


,count,mean,sem,min,max,median,var,std,skew
Regional indicator,,,,,,,,,
North America and ANZ,4,7.173525,0.079830,6.9396,7.2996,7.22745,0.025492,0.159661,-1.715965
Western Europe,21,6.899219,0.149089,5.5150,7.8087,7.09370,0.466778,0.683212,-0.789893
Latin America and Caribbean,20,5.971280,0.151060,3.7208,7.1214,6.05590,0.456383,0.675561,-1.939372
Central and Eastern Europe,16,5.929069,0.126287,4.8827,6.9109,6.01110,0.255175,0.505149,-0.403089
East Asia,6,5.714850,0.187570,5.1239,6.4554,5.69060,0.211096,0.459451,0.561747
Southeast Asia,9,5.383367,0.218955,4.3080,6.3771,5.35350,0.431473,0.656866,-0.035697
Commonwealth of Independent States,12,5.358342,0.154882,4.5607,6.2576,5.54070,0.287861,0.536527,-0.039573
Middle East and North Africa,17,5.227159,0.239665,3.5274,7.1286,5.00510,0.976465,0.988163,0.470270
South Asia,7,4.475443,0.409667,2.5669,5.6933,4.83280,1.174791,1.083878,-0.947583


In [30]:
# Gather the same statistics for the 2021 report
ladder_stats21 = by_region21['Ladder score'].agg(['count', 'mean', 'sem', 'min', 'max', 'median', 'var', 'std', 'skew',])
ladder_stats21.sort_values('mean',ascending=False)

,count,mean,sem,min,max,median,var,std,skew
Regional indicator,,,,,,,,,
North America and ANZ,4,7.128500,0.069028,6.951,7.277,7.1430,0.019060,0.138057,-0.550372
Western Europe,21,6.914905,0.143264,5.536,7.842,7.0850,0.431018,0.656519,-0.705873
Central and Eastern Europe,16,6.040000,0.112991,5.117,6.965,6.1090,0.204271,0.451963,-0.345669
Latin America and Caribbean,20,5.908050,0.155064,3.615,7.069,5.9920,0.480896,0.693467,-1.942143
East Asia,6,5.810333,0.179594,5.339,6.584,5.7610,0.193524,0.439913,1.131835
Commonwealth of Independent States,12,5.467000,0.126473,4.875,6.179,5.4715,0.191946,0.438116,0.305406
Southeast Asia,9,5.407556,0.202090,4.426,6.377,5.3840,0.367565,0.606271,0.012035
Middle East and North Africa,17,5.219765,0.242356,3.658,7.157,4.8870,0.998519,0.999259,0.643576
Sub-Saharan Africa,36,4.494472,0.109149,3.145,6.049,4.6160,0.428884,0.654892,-0.154367


## Let's look at how these statistics changed from 2020 to 2021

In [31]:
year_diff = ladder_stats21.sub(ladder_stats20)
year_diff = year_diff.drop('count', axis=1)
year_diff.sort_values(by='mean',ascending=False)
year_diff.style.set_caption("Change in Report's Summary Statistics from 2020 to 2021")

,mean,sem,min,max,median,var,std,skew
Regional indicator,,,,,,,,
Central and Eastern Europe,0.110931,-0.013296,0.234300,0.054100,0.097900,-0.050904,-0.053185,0.057420
Commonwealth of Independent States,0.108658,-0.028409,0.314300,-0.078600,-0.069200,-0.095915,-0.098411,0.344979
East Asia,0.095483,-0.007976,0.215100,0.128600,0.070400,-0.017572,-0.019538,0.570088
Latin America and Caribbean,-0.063230,0.004004,-0.105800,-0.052400,-0.063900,0.024514,0.017906,-0.002770
Middle East and North Africa,-0.007394,0.002691,0.130600,0.028400,-0.118100,0.022053,0.011097,0.173306
North America and ANZ,-0.045025,-0.010802,0.011400,-0.022600,-0.084450,-0.006432,-0.021604,1.165594
South Asia,-0.033586,-0.034174,-0.043900,-0.424300,0.101200,-0.187825,-0.090416,-0.478518
Southeast Asia,0.024189,-0.016865,0.118000,-0.000100,0.030500,-0.063908,-0.050595,0.047732
Sub-Saharan Africa,0.040228,0.002747,-0.154200,-0.052300,0.051500,0.021315,0.016481,-0.142507


## We can look at the correlation between the ladder score per region with any of the happiness influences.

In [32]:
# Group the data by region

factors = ['Logged GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
LS_factors = ['Ladder score']+factors
ladder_factors20 = data20.groupby(['Regional indicator'])[LS_factors]
ladder_factors21 = data21.groupby(['Regional indicator'])[LS_factors]

#Get the sample mean and variance for each happiness factor and ladder score for each region
means20 = ladder_factors20.mean(); means20.sort_values(by = 'Ladder score')
means21 = ladder_factors21.mean(); means21.sort_values(by = 'Ladder score')
vars20 = ladder_factors20.var(); vars20.sort_values(by = 'Ladder score')
vars21 = ladder_factors20.var(); vars21.sort_values(by = 'Ladder score')

means20.style.set_caption("Sample Mean of Seven Variables from the 2020 Report")

,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Regional indicator,,,,,,,
Central and Eastern Europe,5.929069,10.004773,0.877952,68.189273,0.772235,-0.107890,0.847944
Commonwealth of Independent States,5.358342,9.207234,0.856729,64.726558,0.784004,-0.041285,0.746387
East Asia,5.714850,10.316895,0.859852,71.094847,0.761027,-0.051978,0.702266
Latin America and Caribbean,5.971280,9.255175,0.853971,66.878195,0.829322,-0.075523,0.795991
Middle East and North Africa,5.227159,9.713648,0.796720,65.313611,0.710171,-0.084426,0.761510
North America and ANZ,7.173525,10.709919,0.933842,72.177067,0.907045,0.164180,0.431717
South Asia,4.475443,8.559491,0.708996,62.448662,0.771805,0.035106,0.790656
Southeast Asia,5.383367,9.366847,0.824093,64.710476,0.912667,0.161870,0.704920
Sub-Saharan Africa,4.454244,7.986483,0.695877,55.540890,0.729871,-0.002490,0.766143


In [33]:
means21.style.set_caption("Sample Mean of Seven Variables from the 2021 Report")

,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Regional indicator,,,,,,,
Central and Eastern Europe,6.040000,10.135063,0.892562,68.517437,0.799937,-0.086250,0.847125
Commonwealth of Independent States,5.467000,9.401833,0.872500,65.009500,0.816917,-0.036000,0.725083
East Asia,5.810333,10.367667,0.860500,71.252167,0.763500,-0.062333,0.683333
Latin America and Caribbean,5.908050,9.370000,0.839500,67.076050,0.831750,-0.067700,0.792600
Middle East and North Africa,5.219765,9.666118,0.797647,65.609118,0.716471,-0.079765,0.762235
North America and ANZ,7.128500,10.809500,0.933500,72.325000,0.898750,0.120000,0.449250
South Asia,4.441857,8.682571,0.703429,62.681000,0.765000,0.042714,0.797429
Southeast Asia,5.407556,9.421444,0.820333,64.888444,0.909000,0.156333,0.709111
Sub-Saharan Africa,4.494472,8.075194,0.696750,55.886472,0.723194,0.013444,0.765944


In [34]:
vars20.style.set_caption("Sample Variance of Seven Variables from the 2020 Report")

,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Regional indicator,,,,,,,
Central and Eastern Europe,0.255175,0.153304,0.004773,2.863981,0.007412,0.013330,0.008534
Commonwealth of Independent States,0.287861,0.499737,0.008294,1.309418,0.009539,0.026143,0.022947
East Asia,0.211096,0.382400,0.003061,27.307882,0.009653,0.022551,0.023739
Latin America and Caribbean,0.456383,0.423239,0.005221,10.395802,0.009036,0.017742,0.004898
Middle East and North Africa,0.976465,0.859197,0.006558,14.009630,0.017764,0.015973,0.005887
North America and ANZ,0.025492,0.030228,0.000261,6.718092,0.001931,0.001067,0.039370
South Asia,1.174791,0.563129,0.022089,35.799174,0.030421,0.006489,0.007996
Southeast Asia,0.431473,0.943737,0.003116,32.799713,0.000704,0.058738,0.058262
Sub-Saharan Africa,0.407568,0.714782,0.009641,14.881232,0.007637,0.015198,0.014351


In [35]:
vars21.style.set_caption("Sample Variance of Seven Variables from the 2021 Report")

,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Regional indicator,,,,,,,
Central and Eastern Europe,0.255175,0.153304,0.004773,2.863981,0.007412,0.013330,0.008534
Commonwealth of Independent States,0.287861,0.499737,0.008294,1.309418,0.009539,0.026143,0.022947
East Asia,0.211096,0.382400,0.003061,27.307882,0.009653,0.022551,0.023739
Latin America and Caribbean,0.456383,0.423239,0.005221,10.395802,0.009036,0.017742,0.004898
Middle East and North Africa,0.976465,0.859197,0.006558,14.009630,0.017764,0.015973,0.005887
North America and ANZ,0.025492,0.030228,0.000261,6.718092,0.001931,0.001067,0.039370
South Asia,1.174791,0.563129,0.022089,35.799174,0.030421,0.006489,0.007996
Southeast Asia,0.431473,0.943737,0.003116,32.799713,0.000704,0.058738,0.058262
Sub-Saharan Africa,0.407568,0.714782,0.009641,14.881232,0.007637,0.015198,0.014351


In [36]:
# Calculate how the average ladder score and average factor scores changed over the year for each region

dladder = means21[['Ladder score']] - means20[['Ladder score']]
dfactor = means21[factors] - means20[factors]

dladder.style.set_caption('Change in Average Ladder Score between the 2020 and 2021 Reports')


,Ladder score
Regional indicator,
Central and Eastern Europe,0.110931
Commonwealth of Independent States,0.108658
East Asia,0.095483
Latin America and Caribbean,-0.063230
Middle East and North Africa,-0.007394
North America and ANZ,-0.045025
South Asia,-0.033586
Southeast Asia,0.024189
Sub-Saharan Africa,0.040228


In [37]:
dfactor = means21[factors] - means20[factors]
dfactor.style.set_caption(f"Difference of Means for the Report's Proposed Happpiness Factors")

,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Regional indicator,,,,,,
Central and Eastern Europe,0.130290,0.014610,0.328164,0.027702,0.021640,-0.000819
Commonwealth of Independent States,0.194600,0.015771,0.282942,0.032913,0.005285,-0.021304
East Asia,0.050772,0.000648,0.157320,0.002473,-0.010356,-0.018933
Latin America and Caribbean,0.114825,-0.014471,0.197855,0.002428,0.007823,-0.003391
Middle East and North Africa,-0.047530,0.000927,0.295506,0.006300,0.004662,0.000726
North America and ANZ,0.099581,-0.000342,0.147933,-0.008295,-0.044180,0.017533
South Asia,0.123081,-0.005567,0.232338,-0.006805,0.007608,0.006773
Southeast Asia,0.054597,-0.003760,0.177969,-0.003667,-0.005536,0.004191
Sub-Saharan Africa,0.088711,0.000873,0.345582,-0.006677,0.015934,-0.000198


# Time to Make a Ton of Scatter Plots
### Were interested in any correlation we see between regions in how their happiness facotrs changed. 

In [38]:
colr = ['bright blue', 'bright violet', 'pale red', 'bright aqua', 'bright olive', 'bright teal', 'melon', 'lime', 'bright yellow', 'bright orange']
bck_clr = ['ivory', 'very light green', 'cream', 'very light blue', 'eggshell', 'very light purple',  'off white', 'light peach', 'creme', 'light olive']
xkcd = ['xkcd:'+colr[i] for i in range(10)]
bck_xkcd = ['xkcd:'+bck_clr[i] for i in range(10)]
sizes = ladder_stats21['count']
region_counts = [f'{R} \n (n= {by_region21["Ladder score"].agg("count")[R]})' for R in region_list]

fig, axs = plt.subplots(2,3,figsize=(25,10),facecolor='xkcd:light periwinkle')
ax_list = axs.tolist()[0] + axs.tolist()[1]

x = means20['Ladder score'].values.tolist()
y = dict.fromkeys(factors)
for i in range(6):
    y[factors[i]] = means20[factors[i]].values.tolist()
u = [dladder.values.tolist()[k][0] for k in range(10)]
v = [dfactor.values.tolist()[k][0] for k in range(10)]
z = dict.fromkeys(factors)
for i in range(6):
    z[factors[i]] = [(x[j],y[factors[i]][j],u[j],v[j]) for j in range(10)]
    z[factors[i]].sort(key=lambda x: x[0])

xs = {i: [z[factors[i]][j][0] for j in range(10)] for i in range(6)}
ys = {i: [z[factors[i]][j][1] for j in range(10)] for i in range(6)}
us = {i: [z[factors[i]][j][2] for j in range(10)] for i in range(6)}
vs = {i: [z[factors[i]][j][3] for j in range(10)] for i in range(6)}

res20 = dict.fromkeys(range(6))
lines20 = dict.fromkeys(range(6))

for i in range(6):
    res20[i] = st.linregress(xs[i],ys[i])
    lines20[i] = [res20[i].slope*x0 + res20[i].intercept for x0 in xs[i]]

region_order = means20['Ladder score'].sort_values().index.tolist()

weights = [2*sizes[R]/np.mean(sizes.values) for R in region_order]
weights = np.multiply(weights,125)

corr = dict.fromkeys(range(6))
for i in range(6):
    corr[i] = (round(st.spearmanr(xs[i],ys[i])[0],4), round(st.spearmanr(xs[i],ys[i])[1],4))

for i in range(6):
    ax_list[i].set_xlabel("Mean Ladder Score")
    ax_list[i].set_ylabel(f"Mean {factors[i]}")
    ax_list[i].set_facecolor(bck_xkcd[i])
    ax_list[i].quiver(xs[i],ys[i],us[i],vs[i], label='Velocity')
    ax_list[i].set_title(f"{factors[i]} \n and it's Relationship to a Region's Happiness \n y = {round(res20[i].slope,2)}*x + {round(res20[i].intercept,2)}, r2 = {round(res20[i].rvalue,4)}, p = {round(res20[i].pvalue,4)} \n Spearman's Correlation Coefficient and p-value: {corr[i]}")
    for j in range(10):
        ax_list[i].scatter(xs[i][j],ys[i][j],color=xkcd[j],label=region_order[j],s=125)
    ax_list[i].plot(xs[i],lines20[i], color = 'red', linestyle='dashed')

box = ax_list[4].get_position()
ax_list[4].set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])
ax_list[4].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),ncol=5)

plt.subplots_adjust(wspace=0.35,hspace=0.5)
plt.savefig(fname='ladder_score_factors20.png')
plt.close('all')

In [39]:
normed20 = dict.fromkeys(region_list)
normed21 = dict.fromkeys(region_list)
kde20 = dict.fromkeys(region_list)
kde21 = dict.fromkeys(region_list)
col_ord = range(10)
rev_ord = [9,8,7,6,5,4,3,2,1,0]

for R in region_list:
    mu1 = by_region20.get_group(R)['Ladder score'].mean()
    sig1 = by_region20.get_group(R)['Ladder score'].std()
    minus = by_region20.get_group(R)['Ladder score'].sub(mu1)
    normed20[R] = minus.div(sig1)
    kde20[R] = st.gaussian_kde(normed20[R])(np.linspace(-3,3,500))
    
    mu2 = by_region21.get_group(R)['Ladder score'].mean()
    sig2 = by_region21.get_group(R)['Ladder score'].std()
    minus = by_region21.get_group(R)['Ladder score'].sub(mu2)
    normed21[R] = minus.div(sig2)
    kde21[R] = st.gaussian_kde(normed21[R])(np.linspace(-3,3,500))

In [40]:

ladder_scatter = dict.fromkeys(region_list)
R_file = [R.replace(' ', '_') for R in region_list]
file_names = {i: f'{R_file[i]}_ladder_scatter.png' for i in range(10)}

for R in region_list:
    fig, ax = plt.subplots(figsize=(6,6),facecolor='xkcd:light periwinkle')
    ax.set_facecolor('xkcd:cream')
    xmin, xmax = ax.get_xticks()[0], ax.get_xticks()[-1]
    ymin, ymax = ax.get_yticks()[0], ax.get_yticks()[-1]
    best_fit = st.linregress(normed20[R],normed21[R])
    fnc = [best_fit.slope*x + best_fit.intercept for x in normed20[R]]
    ladder_scatter[R] = [ax.scatter(normed20[R],normed21[R],s=120,c=normed20[R],cmap='magma'),ax.plot(normed20[R],fnc,linestyle='dashed',lw = 2)]
    text = ax.set_title(f'2020 vs. 2021 Reported Ladder Score in {R} \n (y = {round(best_fit.slope,2)}*x + {round(best_fit.intercept,2)}, r2 = {round(best_fit.rvalue,4)}, p = {round(best_fit.pvalue,4)})',fontsize=10,horizontalalignment='center')
    spaced = plt.subplots_adjust(wspace=.2,hspace=0.5)
    saved = plt.savefig(fname=file_names[region_list.index(R)])
    plt.close('all')


In [41]:
fig, axs = plt.subplots(2,5,figsize=(25,10),facecolor='xkcd:light periwinkle')
ax_list2 = axs.tolist()[0] + axs.tolist()[1]

for i in range(10):
    ax_list2[i].set_xlim(-3,3)
    ax_list2[i].set_xticks([-3,-2,-1,0,1,2,3])
    ax_list2[i].plot(np.linspace(-3,3,500),kde20[region_list[i]],color=xkcd[col_ord[i]],label='2020')
    ax_list2[i].fill_between(np.linspace(-3,3,500),kde20[region_list[i]], color = xkcd[col_ord[i]],alpha=0.5)
    ax_list2[i].plot(np.linspace(-3,3,500),kde21[region_list[i]],color=xkcd[rev_ord[i]],label='2021')
    ax_list2[i].fill_between(np.linspace(-3,3,500),kde21[region_list[i]], color = xkcd[rev_ord[i]],alpha=0.5)
    ax_list2[i].set_title(f'Changes in Ladder Score Distributions \n by {region_list[i]} \n from 2020 to 2021')
    ax_list2[i].set_facecolor(bck_xkcd[i])
    ax_list2[i].legend(loc="lower left")

plt.subplots_adjust(wspace=.55,hspace=0.3)
plt.savefig(fname='ladder_score_distribution_changes.png')
plt.close('all')

In [42]:
fig, axs = plt.subplots(2,5,figsize=(25,10),facecolor='xkcd:light periwinkle')
ax_list3 = axs.tolist()[0] + axs.tolist()[1]

for i in range(10):
    ax_list3[i].set_xlim(-1,1)
    ax_list3[i].set_xticks([-1,-.5,0,.5,1])
    ax_list3[i].plot(np.linspace(-3,3,500),kde20[region_list[i]],color=xkcd[col_ord[i]],label='2020')
    ax_list3[i].fill_between(np.linspace(-3,3,500),kde20[region_list[i]], color = xkcd[col_ord[i]],alpha=0.5)
    ax_list3[i].plot(np.linspace(-3,3,500),kde21[region_list[i]],color=xkcd[rev_ord[i]],label='2021')
    ax_list3[i].fill_between(np.linspace(-3,3,500),kde21[region_list[i]], color = xkcd[rev_ord[i]],alpha=0.5)
    ax_list3[i].set_title(f'Changes in Ladder Score Distributions \n by {region_list[i]} \n from 2020 to 2021')
    ax_list3[i].set_facecolor(bck_xkcd[i])
    ax_list3[i].legend(loc="lower left")

plt.subplots_adjust(wspace=.55,hspace=0.3)
plt.savefig(fname='ls_distribution_zoom.png')
plt.close('all')

In [43]:
ls_ks_dict = {R: [st.ks_2samp(normed21[R],normed20[R])[0],st.ks_2samp(normed21[R],normed20[R])[1]] for R in region_list}
print(f'The null hypothesis of the K-S test states that both samples come from the same distribution.')
ls_ks_df = pd.DataFrame(ls_ks_dict,index = pd.Index(['Statistic','p-value'],name = 'Kolmogorov–Smirnov test'))
ls_ks_df.style

The null hypothesis of the K-S test states that both samples come from the same distribution.


,Western Europe,North America and ANZ,Middle East and North Africa,Latin America and Caribbean,Central and Eastern Europe,East Asia,Southeast Asia,Commonwealth of Independent States,Sub-Saharan Africa,South Asia
Kolmogorov–Smirnov test,,,,,,,,,,
Statistic,0.095238,0.250000,0.176471,0.150000,0.125000,0.333333,0.111111,0.250000,0.083333,0.142857
p-value,0.999996,1.000000,0.963107,0.983137,0.999891,0.930736,1.000000,0.868982,0.999774,0.999961


In [44]:
ls_pearson_dict = {R: [st.pearsonr(normed21[R],normed20[R])[0],st.pearsonr(normed21[R],normed20[R])[1]] for R in region_list}
ls_pearson_df = pd.DataFrame(ls_pearson_dict, index = pd.Index(['Statistic','p-value'],name="Pearson's Correlation Coefficient (Effect Size)"))
print(f"The null hypothesis of Pearson's test states that the true correlation coefficient between the ladder scores in 2020 and 2021 is zero, i.e. the samples do not correlate.")
ls_pearson_df.style

The null hypothesis of Pearson's test states that the true correlation coefficient between the ladder scores in 2020 and 2021 is zero, i.e. the samples do not correlate.


,Western Europe,North America and ANZ,Middle East and North Africa,Latin America and Caribbean,Central and Eastern Europe,East Asia,Southeast Asia,Commonwealth of Independent States,Sub-Saharan Africa,South Asia
Pearson's Correlation Coefficient (Effect Size),,,,,,,,,,
Statistic,0.996669,0.940309,0.992086,0.997922,0.991841,0.980398,0.995568,0.970648,0.993875,0.983886
p-value,0.000000,0.059691,0.000000,0.000000,0.000000,0.000573,0.000000,0.000000,0.000000,0.000063


## According Pearson's test, we can reject the hypothesis that the ladder scores between the years are not correlated for every region, except perhaps North America and ANZ.

## Now, we'll look at how influencial each factor is in determining ladder score.

In [45]:
explain20, explain21 = dict.fromkeys(region_list), dict.fromkeys(region_list)
categories = ['Explained by: Log GDP per capita','Explained by: Social support','Explained by: Healthy life expectancy','Explained by: Freedom to make life choices','Explained by: Generosity','Explained by: Perceptions of corruption']
stats = ['mean','var']

for R in region_list:
    explain20[R] = [[round(by_region20.get_group(R)[categories].agg(['mean','var']).loc['mean'].values.tolist()[i],4) for i in range(6)],
    [round(by_region20.get_group(R)[categories].agg(['mean','var']).loc['var'].values.tolist()[i],4) for i in range(6)]]
    explain21[R] = [[round(by_region21.get_group(R)[categories].agg(['mean','var']).loc['mean'].values.tolist()[i],4) for i in range(6)],
    [round(by_region21.get_group(R)[categories].agg(['mean','var']).loc['var'].values.tolist()[i],4) for i in range(6)]]

exp20_list = list(explain20.values())
exp21_list = list(explain21.values())
exp20_arr = np.array(exp20_list[0])
exp21_arr = np.array(exp21_list[0])

for i in range(1,10):
    exp20_arr = np.append(exp20_arr,exp20_list[i],axis=0)
    exp21_arr = np.append(exp21_arr,exp21_list[i],axis=0)

exp_index = pd.MultiIndex.from_product([region_list,stats],names=['Regional indicator','Statistic'])
exp_columns = pd.Index(categories)

exp20_df = pd.DataFrame(exp20_arr,index=exp_index,columns=exp_columns)
exp21_df = pd.DataFrame(exp21_arr,index=exp_index,columns=exp_columns)


In [46]:
exp20_df.head()

Explained by: Log GDP per capita  \
Regional indicator           Statistic                                     
Western Europe               mean                                 1.3004   
                             var                                  0.0090   
North America and ANZ        mean                                 1.3071   
                             var                                  0.0029   
Middle East and North Africa mean                                 0.9983   

                                        Explained by: Social support  \
Regional indicator           Statistic                                 
Western Europe               mean                             1.4022   
                             var                              0.0124   
North America and ANZ        mean                             1.4511   
                             var                              0.0015   
Middle East and North Africa mean                             1.1273   

                                        Explained by: Healthy life expectancy  \
Regional indicator           Statistic                                          
Western Europe               mean                                      0.9959   
                             var                                       0.0007   
North America and ANZ        mean                                      0.9712   
                             var                                       0.0087   
Middle East and North Africa mean                                      0.7241   

                                        Explained by: Freedom to make life choices  \
Regional indicator           Statistic                                               
Western Europe               mean                                           0.5494   
                             var                                            0.0159   
North America and ANZ        mean                                           0.6118   
                             var                                            0.0028   
Middle East and North Africa mean                                           0.3759   

                                        Explained by: Generosity  \
Regional indicator           Statistic                             
Western Europe               mean                         0.2248   
                             var                          0.0097   
North America and ANZ        mean                         0.3076   
                             var                          0.0005   
Middle East and North Africa mean                         0.1432   

                                        Explained by: Perceptions of corruption  
Regional indicator           Statistic                                           
Western Europe               mean                                        0.2627  
                             var                                         0.0236  
North America and ANZ        mean                                        0.3253  
                             var                                         0.0164  
Middle East and North Africa mean                                        0.1124

## To see if the difference in means between regions is significant at all by using ANOVA, we must look at the similarities between the variances, and the normality of the scores.

In [47]:
exp20_var = dict.fromkeys(region_list)

for R in region_list:
    exp20_var[R] = exp20_df.loc[R, 'var'].values.tolist()
reg_combos = [pair for pair in it.combinations(region_list,2)]
combo_keys = [trip for trip in [(reg_combos[i][0],reg_combos[i][1],categories[j]) for i in range(45) for j in range(6)]]
front_reg = [reg_combos[i][0] for i in range(45)]
back_reg = [reg_combos[i][1] for i in range(45)]
pairvar20 = [[exp20_var[front_reg[i]][j],exp20_var[back_reg[i]][j]] for i in range(45) for j in range(6)]
max_var20 = [max(pairvar20[i]) for i in range(270)]
min_var20 = [min(pairvar20[i]) for i in range(270)]
pairvar_ratio20 = list(np.divide(max_var20,min_var20))

closest_pairvar20 = {combo_keys[i]: pairvar_ratio20[i] for i in range(270) if pairvar_ratio20[i] > 0.95 and pairvar_ratio20[i] < 1.05}
closer_pairvar20 = {combo_keys[i]: pairvar_ratio20[i] for i in range(270) if pairvar_ratio20[i] > 0.75 and pairvar_ratio20[i] < 1.25 and combo_keys[i] not in closest_pairvar20.keys()}
close_pairvar20 = {combo_keys[i]: pairvar_ratio20[i] for i in range(270) if pairvar_ratio20[i] > 0.5 and pairvar_ratio20[i] < 1.5 and combo_keys[i] not in closest_pairvar20.keys() and combo_keys[i] not in closer_pairvar20.keys()}

print(f'There are {len(closest_pairvar20)} region pairs with "Explained By:" variances within +/- 0.05 of 1 in 2020. \n There are {len(closer_pairvar20)+len(closest_pairvar20)} regions pairs with "Explained By:" variances within +/- 0.25 of 1 in 2020. \n There are {len(close_pairvar20)+len(closer_pairvar20)+len(closest_pairvar20)} regions pairs with "Explained By:" variances within +/- 0.5 of 1 in 2020.')

There are 6 region pairs with "Explained By:" variances within +/- 0.05 of 1 in 2020. 
 There are 33 regions pairs with "Explained By:" variances within +/- 0.25 of 1 in 2020. 
 There are 63 regions pairs with "Explained By:" variances within +/- 0.5 of 1 in 2020.


## We should probably restrict our ANOVA to these 63 pairs, as it has been shows that a variance differemnce greater that 0.5 will significantly effect the ANOVA results.

In [48]:
#Calculate the difference in means among these 63 pairs, broken into close, closer, and closest.
closevar20_fronts = [by_region20.get_group(list(close_pairvar20.keys())[i][0])[list(close_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(close_pairvar20))]
closevar20_backs = [by_region20.get_group(list(close_pairvar20.keys())[i][1])[list(close_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(close_pairvar20))]
closevar20_means = [(np.mean(closevar20_fronts[i]),np.mean(closevar20_backs[i])) for i in range(len(closevar20_fronts))]

closervar20_fronts = [by_region20.get_group(list(closer_pairvar20.keys())[i][0])[list(closer_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(closer_pairvar20))]
closervar20_backs = [by_region20.get_group(list(closer_pairvar20.keys())[i][1])[list(closer_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(closer_pairvar20))]
closervar20_means = [(np.mean(closervar20_fronts[i]),np.mean(closervar20_backs[i])) for i in range(len(closervar20_fronts))]

closestvar20_fronts = [by_region20.get_group(list(closest_pairvar20.keys())[i][0])[list(closest_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(closest_pairvar20))]
closestvar20_backs = [by_region20.get_group(list(closest_pairvar20.keys())[i][1])[list(closest_pairvar20.keys())[i][-1]].values.tolist() for i in range(len(closest_pairvar20))]
closestvar20_means = [(np.mean(closestvar20_fronts[i]),np.mean(closestvar20_backs[i])) for i in range(len(closestvar20_fronts))]

close_pairs20 = list(close_pairvar20.keys())+list(closer_pairvar20.keys())+list(closest_pairvar20.keys())
all_means20 = closevar20_means + closervar20_means + closestvar20_means

diff_means20 = {close_pairs20[i]: round(all_means20[i][0] - all_means20[i][1],4) for i in range(len(close_pairs20))}

close_means20 = []
for x in diff_means20.items():
    if x[0] in close_pairvar20.keys():
        close_means20.append((x[0],x[1],close_pairvar20[x[0]]))
    elif x[0] in closer_pairvar20.keys():
        close_means20.append((x[0],x[1],closer_pairvar20[x[0]]))
    else:
        close_means20.append((x[0],x[1],closest_pairvar20[x[0]]))
closest_means20 = {x[0]: (x[1],x[2]) for x in close_means20 if abs(1-abs(x[2]))<0.05 and abs(x[1])<0.05}
print(f'There are two factors between one region pair with very similar means and variances.')
pd.DataFrame(closest_means20,index=['Diff of Means','Ratio of Var']).T

There are two factors between one region pair with very similar means and variances.


Diff of Means  \
East Asia Commonwealth of Independent States Explained by: Freedom to make life choices        -0.0275   
                                             Explained by: Perceptions of corruption            0.0285   

                                                                                         Ratio of Var  
East Asia Commonwealth of Independent States Explained by: Freedom to make life choices      1.014599  
                                             Explained by: Perceptions of corruption         1.031250

## Given the small number of region pairs in 2020, we can raise our significance cutoff for the mean to 0.1.

In [49]:
closest_means20 = {x[0]: (x[1],x[2]) for x in close_means20 if abs(1-abs(x[2]))<0.05 and abs(x[1])<0.1}
pd.DataFrame(closest_means20,index=['Diff of Means','Ratio of Var']).T

Diff of Means  \
Western Europe             East Asia                          Explained by: Generosity                           0.0601   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices         0.0508   
East Asia                  Southeast Asia                     Explained by: Social support                       0.0845   
                           Commonwealth of Independent States Explained by: Freedom to make life choices        -0.0275   
                                                              Explained by: Perceptions of corruption            0.0285   

                                                                                                          Ratio of Var  
Western Europe             East Asia                          Explained by: Generosity                        1.020619  
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices      1.037736  
East Asia                  Southeast Asia                     Explained by: Social support                    1.017544  
                           Commonwealth of Independent States Explained by: Freedom to make life choices      1.014599  
                                                              Explained by: Perceptions of corruption         1.031250

In [50]:
fig, ax = plt.subplots(figsize=(10,10))
fig.set_facecolor('xkcd:parchment')
ax.set_xlabel('Ratio of Variance',fontsize=14)
ax.set_ylabel('Difference of Means',fontsize=14)
joined_vars = list(closest_pairvar20.values()) + list(closer_pairvar20.values()) + list(close_pairvar20.values())
mu_sig_pts20 = [(joined_vars[i],list(diff_means20.values())[i]) for i in range(len(list(joined_vars)))]
sorted(mu_sig_pts20,key = lambda x: np.sqrt((x[0]-1)**2 + x[1]**2))
divnorm=colors.TwoSlopeNorm(vcenter=0)
ax.scatter(joined_vars,list(diff_means20.values()),c=[mu_sig_pts20[i][1] for i in range(len(list(joined_vars)))],norm=divnorm,cmap='Spectral',s=100)
ax.hlines(0,1,1.5,colors='blue',linestyles='dashed')
ax.set_facecolor('xkcd:pastel purple')
ax.set_title(f"Comparing the Ratio of Variance to the \n Difference of Means of the Six Happiness Factors' Influence (2020)",horizontalalignment = 'center', fontsize=16)
plt.savefig('mean_variance_scatter20')
plt.close('all')

In [51]:
exp21_df.head()

Explained by: Log GDP per capita  \
Regional indicator           Statistic                                     
Western Europe               mean                                 1.4629   
                             var                                  0.0113   
North America and ANZ        mean                                 1.4583   
                             var                                  0.0030   
Middle East and North Africa mean                                 1.0589   

                                        Explained by: Social support  \
Regional indicator           Statistic                                 
Western Europe               mean                             1.0180   
                             var                              0.0119   
North America and ANZ        mean                             1.0610   
                             var                              0.0009   
Middle East and North Africa mean                             0.7546   

                                        Explained by: Healthy life expectancy  \
Regional indicator           Statistic                                          
Western Europe               mean                                      0.7735   
                             var                                       0.0006   
North America and ANZ        mean                                      0.7513   
                             var                                       0.0076   
Middle East and North Africa mean                                      0.5395   

                                        Explained by: Freedom to make life choices  \
Regional indicator           Statistic                                               
Western Europe               mean                                           0.5804   
                             var                                            0.0144   
North America and ANZ        mean                                           0.6285   
                             var                                            0.0025   
Middle East and North Africa mean                                           0.4074   

                                        Explained by: Generosity  \
Regional indicator           Statistic                             
Western Europe               mean                         0.1859   
                             var                          0.0078   
North America and ANZ        mean                         0.2662   
                             var                          0.0004   
Middle East and North Africa mean                         0.1358   

                                        Explained by: Perceptions of corruption  
Regional indicator           Statistic                                           
Western Europe               mean                                        0.2655  
                             var                                         0.0220  
North America and ANZ        mean                                        0.3127  
                             var                                         0.0144  
Middle East and North Africa mean                                        0.1130

In [52]:
exp21_var = dict.fromkeys(region_list)

for R in region_list:
    exp21_var[R] = exp21_df.loc[R, 'var'].values.tolist()
reg_combos = [pair for pair in it.combinations(region_list,2)]
combo_keys = [trip for trip in [(reg_combos[i][0],reg_combos[i][1],categories[j]) for i in range(45) for j in range(6)]]
front_reg = [reg_combos[i][0] for i in range(45)]
back_reg = [reg_combos[i][1] for i in range(45)]
pairvar21 = [[exp21_var[front_reg[i]][j],exp21_var[back_reg[i]][j]] for i in range(45) for j in range(6)]
max_var21 = [max(pairvar21[i]) for i in range(270)]
min_var21 = [min(pairvar21[i]) for i in range(270)]
pairvar_ratio21 = list(np.divide(max_var21,min_var21))

closest_pairvar21 = {combo_keys[i]: pairvar_ratio21[i] for i in range(270) if pairvar_ratio21[i] > 0.95 and pairvar_ratio21[i] < 1.05}
closer_pairvar21 = {combo_keys[i]: pairvar_ratio21[i] for i in range(270) if pairvar_ratio21[i] > 0.75 and pairvar_ratio21[i] < 1.25 and combo_keys[i] not in closest_pairvar21.keys()}
close_pairvar21 = {combo_keys[i]: pairvar_ratio21[i] for i in range(270) if pairvar_ratio21[i] > 0.5 and pairvar_ratio21[i] < 1.5 and combo_keys[i] not in closest_pairvar21.keys() and combo_keys[i] not in closer_pairvar21.keys()}

print(f'There are {len(closest_pairvar21)} region pairs with "Explained By:" variances within +/- 0.05 of 1 in 2021. \n There are {len(closer_pairvar21)+len(closest_pairvar21)} regions pairs with "Explained By:" variances within +/- 0.25 of 1 in 2021. \n There are {len(close_pairvar21)+len(closer_pairvar21)+len(closest_pairvar21)} regions pairs with "Explained By:" variances within +/- 0.5 of 1 in 2021.')

There are 11 region pairs with "Explained By:" variances within +/- 0.05 of 1 in 2021. 
 There are 38 regions pairs with "Explained By:" variances within +/- 0.25 of 1 in 2021. 
 There are 66 regions pairs with "Explained By:" variances within +/- 0.5 of 1 in 2021.


In [53]:
closevar21_fronts = [by_region21.get_group(list(close_pairvar21.keys())[i][0])[list(close_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(close_pairvar21))]
closevar21_backs = [by_region21.get_group(list(close_pairvar21.keys())[i][1])[list(close_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(close_pairvar21))]
closevar21_means = [(np.mean(closevar21_fronts[i]),np.mean(closevar21_backs[i])) for i in range(len(closevar21_fronts))]

closervar21_fronts = [by_region21.get_group(list(closer_pairvar21.keys())[i][0])[list(closer_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(closer_pairvar21))]
closervar21_backs = [by_region21.get_group(list(closer_pairvar21.keys())[i][1])[list(closer_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(closer_pairvar21))]
closervar21_means = [(np.mean(closervar21_fronts[i]),np.mean(closervar21_backs[i])) for i in range(len(closervar21_fronts))]

closestvar21_fronts = [by_region21.get_group(list(closest_pairvar21.keys())[i][0])[list(closest_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(closest_pairvar21))]
closestvar21_backs = [by_region21.get_group(list(closest_pairvar21.keys())[i][1])[list(closest_pairvar21.keys())[i][-1]].values.tolist() for i in range(len(closest_pairvar21))]
closestvar21_means = [(np.mean(closestvar21_fronts[i]),np.mean(closestvar21_backs[i])) for i in range(len(closestvar21_fronts))]

close_pairs21 = list(close_pairvar21.keys())+list(closer_pairvar21.keys())+list(closest_pairvar21.keys())
all_means21 = closevar21_means + closervar21_means + closestvar21_means

diff_means21 = {close_pairs21[i]: round(all_means21[i][0] - all_means21[i][1],4) for i in range(len(close_pairs21))}
close_means21 = []
for x in diff_means21.items():
    if x[0] in close_pairvar21.keys():
        close_means21.append((x[0],x[1],close_pairvar21[x[0]]))
    elif x[0] in closer_pairvar21.keys():
        close_means21.append((x[0],x[1],closer_pairvar21[x[0]]))
    else:
        close_means21.append((x[0],x[1],closest_pairvar21[x[0]]))
closest_means21 = {x[0]: (x[1],x[2]) for x in close_means21 if abs(1-abs(x[2]))<0.05 and abs(x[1])<0.05}
print(f'There are 2 factors between 6 region pairs with very similar means and variances.')
pd.DataFrame(closest_means21,index=['Diff of Means','Ratio of Var']).T

There are 2 factors between 6 region pairs with very similar means and variances.


Diff of Means  \
Western Europe               Latin America and Caribbean Explained by: Generosity                        0.0422   
                             East Asia                   Explained by: Generosity                        0.0386   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption         0.0194   
                             South Asia                  Explained by: Perceptions of corruption         0.0224   
Latin America and Caribbean  East Asia                   Explained by: Generosity                       -0.0036   
                             South Asia                  Explained by: Perceptions of corruption         0.0031   

                                                                                                  Ratio of Var  
Western Europe               Latin America and Caribbean Explained by: Generosity                     1.000000  
                             East Asia                   Explained by: Generosity                     1.025641  
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption      1.000000  
                             South Asia                  Explained by: Perceptions of corruption      1.038462  
Latin America and Caribbean  East Asia                   Explained by: Generosity                     1.025641  
                             South Asia                  Explained by: Perceptions of corruption      1.038462

## In the above data frame, we can see those region pairs that have almost identical means and variances when it comes to how a factor explains or influences a country's happiness.

In [54]:
full_pairs20 = dict.fromkeys(list(closest_means20.keys()))
for i in range(len(closest_means20)):
    full_pairs20[list(closest_means20.keys())[i]] = [list(by_region20.get_group(list(closest_means20.items())[i][0][0])[list(closest_means20.items())[i][0][2]].values), list(by_region20.get_group(list(closest_means20.items())[i][0][1])[list(closest_means20.items())[i][0][2]].values)]

full_pairs21 = dict.fromkeys(list(closest_means21.keys()))
for i in range(len(closest_means21)):
    full_pairs21[list(closest_means21.keys())[i]] = [list(by_region21.get_group(list(closest_means21.items())[i][0][0])[list(closest_means21.items())[i][0][2]].values), list(by_region21.get_group(list(closest_means21.items())[i][0][1])[list(closest_means21.items())[i][0][2]].values)]

In [55]:
fig, ax = plt.subplots(figsize=(10,10))
fig.set_facecolor('xkcd:parchment')
ax.set_xlabel('Ratio of Variance',fontsize=14)
ax.set_ylabel('Difference of Means',fontsize=14)
joined_vars = list(closest_pairvar21.values()) + list(closer_pairvar21.values()) + list(close_pairvar21.values())
mu_sig_pts21 = [(joined_vars[i],list(diff_means21.values())[i]) for i in range(len(list(diff_means21.values())))]
sorted(mu_sig_pts20,key = lambda x: np.sqrt((x[0]-1)**2 + x[1]**2))
divnorm=colors.TwoSlopeNorm(vcenter=0)
ax.scatter(joined_vars,list(diff_means21.values()),c=[mu_sig_pts21[i][1] for i in range(len(list(joined_vars)))],norm=divnorm,cmap='Spectral',s=100)
ax.hlines(0,1,1.5,colors='blue',linestyles='dashed')
ax.set_facecolor('xkcd:pastel purple')
ax.set_title(f"Comparing the Ratio of Variance to the \n Difference of Means of the Six Happiness Factors' Influence (2021)",horizontalalignment = 'center', fontsize=16)
plt.savefig('mean_variance_scatter')
plt.close('all')

## We can see from the plots that the data points seem to have gotten closer to the mean bar of zero in 2021.
### This possibly implies that these factors, similarly variable as to how they affect happiness over these two regions' countries, are becoming closer on average as to how they affect happiness. Perhaps this means that these two regions have these factors affect their happiness at a similar magnitude.

## Finally, we can perform a two sample t-test and ANOVA on the difference of means between these regional pairs, now we can safely assume that the population variances are likely equal.

In [56]:
def pooled_sd(samp1,samp2):
    n1,n2 = len(samp1), len(samp2)
    s21, s22 = np.var(samp1), np.var(samp2)
    return np.sqrt(((n1 -1)*s21 + (n2-1)*s22)/(n1+n2-2))

SP20 = dict.fromkeys(list(closest_means20.keys()))
for x in list(closest_means20.keys()):
    SP20[x] = pooled_sd(full_pairs20[x][0],full_pairs20[x][1])

SP21 = dict.fromkeys(list(closest_means21.keys()))
for x in list(closest_means21.keys()):
   SP21[x] = pooled_sd(full_pairs21[x][0],full_pairs21[x][1])

In [57]:
mdeg_free20 = dict.fromkeys(list(full_pairs20.keys()))
for x in list(full_pairs20.keys()):
    mdeg_free20[x] = len(full_pairs20[x][0]) + len(full_pairs20[x][1]) - 2

vdeg_free20 = dict.fromkeys(list(full_pairs20.keys()))
for x in list(full_pairs20.keys()):
    vdeg_free20[x] = (len(full_pairs20[x][0]) - 1, len(full_pairs20[x][1]) - 1)

mCI20 = dict.fromkeys(list(closest_means20.keys()))
for x in list(closest_means20.keys()):
    mCI20[x] = (round(closest_means20[x][0] - st.t.ppf(0.95,mdeg_free20[x])*SP20[x]*np.sqrt(1/len(full_pairs20[x][0]) + 1/len(full_pairs20[x][1])),4), round(closest_means20[x][0] + st.t.ppf(0.95,mdeg_free20[x])*SP20[x]*np.sqrt(1/len(full_pairs20[x][0]) + 1/len(full_pairs20[x][1])),4))

vCI20 = dict.fromkeys(list(closest_means20.keys()))
for x in list(closest_means20.keys()):
    vCI20[x] = (round(closest_means20[x][1]/st.f.ppf(0.05,vdeg_free20[x][0],vdeg_free20[x][1]),4), round(closest_means20[x][1]/st.f.ppf(0.95,vdeg_free20[x][0],vdeg_free20[x][1]),4))

In [58]:
mdeg_free21 = dict.fromkeys(list(full_pairs21.keys()))
for x in list(full_pairs21.keys()):
    mdeg_free21[x] = len(full_pairs21[x][0]) + len(full_pairs21[x][1]) - 2

vdeg_free21 = dict.fromkeys(list(full_pairs21.keys()))
for x in list(full_pairs21.keys()):
    vdeg_free21[x] = (len(full_pairs21[x][0]) - 1, len(full_pairs21[x][1]) - 1)

mCI21 = dict.fromkeys(list(closest_means21.keys()))
for x in list(closest_means21.keys()):
    mCI21[x] = (round(closest_means21[x][0] - st.t.ppf(0.95,mdeg_free21[x])*SP21[x]*np.sqrt(1/len(full_pairs21[x][0]) + 1/len(full_pairs21[x][1])),4), round(closest_means21[x][0] + st.t.ppf(0.95,mdeg_free21[x])*SP21[x]*np.sqrt(1/len(full_pairs21[x][0]) + 1/len(full_pairs21[x][1])),4))

vCI21 = dict.fromkeys(list(closest_means21.keys()))
for x in list(closest_means21.keys()):
    vCI21[x] = (round(closest_means21[x][1]/st.f.ppf(0.95,vdeg_free21[x][0],vdeg_free21[x][1]),4),round(closest_means21[x][1]/st.f.ppf(0.05,vdeg_free21[x][0],vdeg_free21[x][1]),4))

In [59]:
ttest_ANOVA20 = dict.fromkeys(full_pairs20.keys())
for x in full_pairs20.keys(): 
    ttest_ANOVA20[x] = [st.ttest_ind(full_pairs20[x][0],full_pairs20[x][1])[0], st.f_oneway(full_pairs20[x][0],full_pairs20[x][1])[0], st.ttest_ind(full_pairs20[x][0],full_pairs20[x][1])[1]]

ttest_ANOVA21 = dict.fromkeys(full_pairs21.keys())
for x in full_pairs21.keys(): 
    ttest_ANOVA21[x] = [st.ttest_ind(full_pairs21[x][0],full_pairs21[x][1])[0], st.f_oneway(full_pairs21[x][0],full_pairs21[x][1])[0], st.ttest_ind(full_pairs21[x][0],full_pairs21[x][1])[1]]

In [60]:
CI_means20 = {x[0]: (x[1], mCI20[x[0]],x[2], vCI20[x[0]]) for x in close_means20 if abs(1-abs(x[2]))<0.05 and abs(x[1])<0.1}
print('2020')
pd.DataFrame(CI_means20,index=['Diff of Means','95% CI','Ratio of Var','95% CI']).T

2020


Diff of Means  \
Western Europe             East Asia                          Explained by: Generosity                          0.0601   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices        0.0508   
East Asia                  Southeast Asia                     Explained by: Social support                      0.0845   
                           Commonwealth of Independent States Explained by: Freedom to make life choices       -0.0275   
                                                              Explained by: Perceptions of corruption           0.0285   

                                                                                                                     95% CI  \
Western Europe             East Asia                          Explained by: Generosity                    (-0.0149, 0.1351)   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices  (-0.0007, 0.1023)   
East Asia                  Southeast Asia                     Explained by: Social support                (-0.0297, 0.1987)   
                           Commonwealth of Independent States Explained by: Freedom to make life choices  (-0.1241, 0.0691)   
                                                              Explained by: Perceptions of corruption     (-0.0525, 0.1095)   

                                                                                                         Ratio of Var  \
Western Europe             East Asia                          Explained by: Generosity                       1.020619   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices     1.037736   
East Asia                  Southeast Asia                     Explained by: Social support                   1.017544   
                           Commonwealth of Independent States Explained by: Freedom to make life choices     1.014599   
                                                              Explained by: Perceptions of corruption         1.03125   

                                                                                                                    95% CI  
Western Europe             East Asia                          Explained by: Generosity                    (2.7668, 0.2239)  
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices  (2.3066, 0.5287)  
East Asia                  Southeast Asia                     Explained by: Social support                (4.9029, 0.2759)  
                           Commonwealth of Independent States Explained by: Freedom to make life choices  (4.7726, 0.3167)  
                                                              Explained by: Perceptions of corruption      (4.851, 0.3219)

In [61]:
pd.DataFrame(ttest_ANOVA20,index=['T-Statistic','F-Statistic','p-value']).T

T-Statistic  \
Western Europe             East Asia                          Explained by: Generosity                       1.318645   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices     1.620601   
East Asia                  Southeast Asia                     Explained by: Social support                   1.219615   
                           Commonwealth of Independent States Explained by: Freedom to make life choices    -0.469641   
                                                              Explained by: Perceptions of corruption        0.579406   

                                                                                                          F-Statistic  \
Western Europe             East Asia                          Explained by: Generosity                       1.738826   
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices     2.626347   
East Asia                  Southeast Asia                     Explained by: Social support                   1.487460   
                           Commonwealth of Independent States Explained by: Freedom to make life choices     0.220563   
                                                              Explained by: Perceptions of corruption        0.335711   

                                                                                                           p-value  
Western Europe             East Asia                          Explained by: Generosity                    0.199241  
Central and Eastern Europe Sub-Saharan Africa                 Explained by: Freedom to make life choices  0.111394  
East Asia                  Southeast Asia                     Explained by: Social support                0.244277  
                           Commonwealth of Independent States Explained by: Freedom to make life choices  0.644948  
                                                              Explained by: Perceptions of corruption     0.570384

In [62]:
CI_means21 = {x[0]: (x[1], mCI21[x[0]],x[2], vCI21[x[0]]) for x in close_means21 if abs(1-abs(x[2]))<0.05 and abs(x[1])<0.05}
print('2021')
pd.DataFrame(CI_means21,index=['Diff of Means','95% CI','Ratio of Var','95% CI']).T

2021


Diff of Means  \
Western Europe               Latin America and Caribbean Explained by: Generosity                       0.0422   
                             East Asia                   Explained by: Generosity                       0.0386   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption        0.0194   
                             South Asia                  Explained by: Perceptions of corruption        0.0224   
Latin America and Caribbean  East Asia                   Explained by: Generosity                      -0.0036   
                             South Asia                  Explained by: Perceptions of corruption        0.0031   

                                                                                                             95% CI  \
Western Europe               Latin America and Caribbean Explained by: Generosity                 (-0.0032, 0.0876)   
                             East Asia                   Explained by: Generosity                  (-0.0288, 0.106)   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption  (-0.0085, 0.0473)   
                             South Asia                  Explained by: Perceptions of corruption  (-0.0157, 0.0605)   
Latin America and Caribbean  East Asia                   Explained by: Generosity                 (-0.0715, 0.0643)   
                             South Asia                  Explained by: Perceptions of corruption  (-0.0341, 0.0403)   

                                                                                                 Ratio of Var  \
Western Europe               Latin America and Caribbean Explained by: Generosity                         1.0   
                             East Asia                   Explained by: Generosity                    1.025641   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption          1.0   
                             South Asia                  Explained by: Perceptions of corruption     1.038462   
Latin America and Caribbean  East Asia                   Explained by: Generosity                    1.025641   
                             South Asia                  Explained by: Perceptions of corruption     1.038462   

                                                                                                            95% CI  
Western Europe               Latin America and Caribbean Explained by: Generosity                  (0.4639, 2.137)  
                             East Asia                   Explained by: Generosity                  (0.225, 2.7804)  
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption   (0.4515, 2.288)  
                             South Asia                  Explained by: Perceptions of corruption  (0.2648, 2.8467)  
Latin America and Caribbean  East Asia                   Explained by: Generosity                 (0.2245, 2.8103)  
                             South Asia                  Explained by: Perceptions of corruption  (0.2673, 2.7294)

In [63]:
pd.DataFrame(ttest_ANOVA21,index=['T-Statistic','F-Statistic','p-value']).T

T-Statistic  \
Western Europe               Latin America and Caribbean Explained by: Generosity                    1.528491   
                             East Asia                   Explained by: Generosity                    0.941693   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption     1.140397   
                             South Asia                  Explained by: Perceptions of corruption     0.968046   
Latin America and Caribbean  East Asia                   Explained by: Generosity                   -0.088028   
                             South Asia                  Explained by: Perceptions of corruption     0.136037   

                                                                                                  F-Statistic  \
Western Europe               Latin America and Caribbean Explained by: Generosity                    2.336285   
                             East Asia                   Explained by: Generosity                    0.886785   
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption     1.300506   
                             South Asia                  Explained by: Perceptions of corruption     0.937114   
Latin America and Caribbean  East Asia                   Explained by: Generosity                    0.007749   
                             South Asia                  Explained by: Perceptions of corruption     0.018506   

                                                                                                   p-value  
Western Europe               Latin America and Caribbean Explained by: Generosity                 0.134462  
                             East Asia                   Explained by: Generosity                 0.355361  
Middle East and North Africa Latin America and Caribbean Explained by: Perceptions of corruption  0.261865  
                             South Asia                  Explained by: Perceptions of corruption  0.343544  
Latin America and Caribbean  East Asia                   Explained by: Generosity                 0.930585  
                             South Asia                  Explained by: Perceptions of corruption  0.892882

## First of all, the only pair with significant similarities in mean and variance in both reports is Western Europe and East Asia, with how Generosity influences their happiness score. In fact, this correlation seems to grow more significant into 2021, with the p-value increasing and the statistics decreasing. However, we also see that we could reject these similarities as merely chance pretty confidently in 2020.
### Second of all, we cannot reject the null hypothesis that these similarities are just due to chance with a confidence of 95% for any of the pairings. Here are the probable candidates for rejection, however:
#### Central and Eastern Europe and	Sub-Saharan Africa, Freedom to make life choices; Rejected with 89% confidence in 2020
#### Western Europe	and East Asia, Generosity:R ejected with 81% confidence in 2020
#### Latin America and the Caribbean and Western Europe, Generosity; Rejected with 87% confidence in 2021


## Finally, we will directly compare the magnitude of the happiness factors in each region using the normalized data and radar plots.

In [64]:
factors20 = {R: by_region20.get_group(R)[['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']].agg('mean') for R in region_list}
factors21 = {R: by_region21.get_group(R)[['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']].agg('mean') for R in region_list}
factors20_df = pd.DataFrame(factors20,index = ['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']).T
factors21_df = pd.DataFrame(factors21,index = ['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']).T
factors20_df

,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
Western Europe,10.688402,0.913134,72.863565,0.854993,0.038923,0.528730
North America and ANZ,10.709919,0.933842,72.177067,0.907045,0.164180,0.431717
Middle East and North Africa,9.713648,0.796720,65.313611,0.710171,-0.084426,0.761510
Latin America and Caribbean,9.255175,0.853971,66.878195,0.829322,-0.075523,0.795991
Central and Eastern Europe,10.004773,0.877952,68.189273,0.772235,-0.107890,0.847944
East Asia,10.316895,0.859852,71.094847,0.761027,-0.051978,0.702266
Southeast Asia,9.366847,0.824093,64.710476,0.912667,0.161870,0.704920
Commonwealth of Independent States,9.207234,0.856729,64.726558,0.784004,-0.041285,0.746387
Sub-Saharan Africa,7.986483,0.695877,55.540890,0.729871,-0.002490,0.766143
South Asia,8.559491,0.708996,62.448662,0.771805,0.035106,0.790656


In [65]:
factors20_cs = {R: by_region20.get_group(R)[['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']] for R in region_list}
factors21_cs = {R: by_region21.get_group(R)[['Logged GDP per capita','Social support','Healthy life expectancy','Freedom to make life choices','Generosity','Perceptions of corruption']] for R in region_list}
factors20cs_df = dict.fromkeys(region_list)
factors21cs_df = dict.fromkeys(region_list)

for R in region_list:
    factors20cs_df[R] = pd.DataFrame(factors20_cs[R])
    factors21cs_df[R] = pd.DataFrame(factors21_cs[R])


In [66]:
def norm_fnc(df,xmin,xmax):
    oldmax = getattr(df,'max')()
    oldmin = getattr(df,'min')()
    oldrange = getattr(oldmax,'subtract')(oldmin)
    newrange = xmax - xmin
    num = getattr(df,'subtract')(oldmin)
    num2 = getattr(num,'multiply')(newrange)
    div = getattr(num2,'divide')(oldrange)
    return getattr(div,'add')(xmin)

In [67]:
norm_exp20 = dict.fromkeys(region_list)
norm_exp21 = dict.fromkeys(region_list)

for R in region_list:
    norm_exp20[R] = norm_fnc(factors20cs_df[R],0,1)
    norm_exp21[R] = norm_fnc(factors21cs_df[R],0,1)

In [68]:
# radar plot per region, showing explanation magnitude
df20, df21 = dict.fromkeys(region_list), dict.fromkeys(region_list)
for R in region_list: 
    df20[R] = norm_exp20[R].agg('mean')
    df21[R] = norm_exp21[R].agg('mean')
colrs = ['b', 'g', 'r', 'c','b', 'g', 'r', 'c', 'm', 'y', 'tab:pink']
def radar_plot(d,i,j):

    #number of variables
    vars = list(d[region_list[i]])
    N = len(vars)

    #repeat first value
    values = d[region_list[i]].values.tolist()
    values.append(values[0])

    #make the angles
    ang = [n*2*np.pi/float(N) for n in range(N)]
    ang.append(ang[0])

    #make plot
    if j == 3:
        ax_dict = plt.subplot(2,j,i - 3, projection='polar')
    elif j==2:
        ax_dict = plt.subplot(2,j,i + 1, projection='polar')

    # set variable axes
    labels = ['1','2','3','4','5','6']
    var_ticks = plt.xticks(ang[:-1], labels, c='k', size=8, fontsize=12)
    ax.xaxis.set_tick_params(pad=4)

    # set polar axes
    plt.yticks(np.linspace(0,1,10)[1:10],labels=['','',0.33,'','',0.67,'','',1],color='k',size=10)
    plt.ylim(0,1)
    plt.title(f'{region_list[i]} \n Happiness Factors\n',fontsize=16)

    #plot data!
    ax_dict.plot(ang,values,colrs[i],lw=2,linestyle='solid')
    ax_dict.fill(ang,values,colrs[i],alpha=0.4)

leg = f'1: Logged GDP per capita, 2: Social support, 3: Healthy life expectancy, \n 4: Freedom to make life choices, 5: Generosity, 6: Perceptions of corruption'

In [69]:
fig = plt.figure(figsize=(12,15))
bg = fig.set_facecolor('xkcd:light periwinkle')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.75)
fig.text(0.5, 0.525, leg, horizontalalignment='center', fontsize=16, verticalalignment='center', bbox=props)
for i in range(0,4):
    radar_plot(df20,i,2)
plt.subplots_adjust(wspace=.4,hspace=0.4)
plt.savefig(f'{region_list[i]}_factors1_20.png')
plt.close('all')

In [70]:
fig = plt.figure(figsize=(15,15))
bg = fig.set_facecolor('xkcd:light periwinkle')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.75)
fig.text(0.5, 0.525, leg, horizontalalignment='center', fontsize=16, verticalalignment='center', bbox=props)
for i in range(4,10):
    radar_plot(df20,i,3)
plt.subplots_adjust(wspace=.4,hspace=0.5)
plt.savefig(f'{region_list[i]}_factors2_20.png')
plt.close('all')